In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib
import os
os.makedirs("models", exist_ok=True)

In [3]:
# load features...

X_train = np.load("../features/X_features.npy")
y_train = np.load("../features/y_labels.npy")

print("Features loaded successfully")
print("X_features shape:", X_train.shape)
print("y_labels shape:", y_train.shape)


Features loaded successfully
X_features shape: (14940, 2048)
y_labels shape: (14940,)


In [4]:
# load test data...
X_test = np.load("../features/X_test.npy")
y_test = np.load("../features/y_test.npy")

print ("Test features:", X_test.shape)
print ("Test labels:", y_test.shape)

Test features: (371, 2048)
Test labels: (371,)


In [5]:
#Feature Scaling - fit for training , transform for testing

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Feature scaling done")

Feature scaling done


In [6]:
# Train KNN model:

# Try different K values (odd) 1-3-5-7-9...
k_values = [1,3,5,7,9,11]
# try different weighting schemes (uniform, distance)
weighting_schemes = ["uniform", "distance"]
# choose the best model and save it
best_accuracy = 0
best_k = None
best_weight = None
best_model = None

for k in k_values:
    for weight in weighting_schemes:
        print(f"Training k-NN with k={k}, weights={weight}")
        knn = KNeighborsClassifier(n_neighbors=k,weights=weight,metric="cosine")

        knn.fit(X_train,y_train)

        y_val = knn.predict(X_test)
        acc = accuracy_score(y_test, y_val)
        print(f"Validation Accuracy: {acc:.4f}")

        if acc >best_accuracy:
            best_accuracy = acc
            best_k = k
            best_weight = weight
            best_model = knn
print("\nBest k-NN Model:")
print("Best k:", best_k)
print("Best weighting:", best_weight)
print("Best validation accuracy:", best_accuracy)

# save best model
joblib.dump(best_model,"D:\Documents\GitHub\Material-Stream-Identification-System\models\knn_best.pkl")

print("Best knn model saved")


<>:35: SyntaxWarning: invalid escape sequence '\D'
<>:35: SyntaxWarning: invalid escape sequence '\D'
C:\Users\77\AppData\Local\Temp\ipykernel_31400\1319634791.py:35: SyntaxWarning: invalid escape sequence '\D'
  joblib.dump(best_model,"D:\Documents\GitHub\Material-Stream-Identification-System\models\knn_best.pkl")


Training k-NN with k=1, weights=uniform
Validation Accuracy: 0.8679
Training k-NN with k=1, weights=distance
Validation Accuracy: 0.8679
Training k-NN with k=3, weights=uniform
Validation Accuracy: 0.8760
Training k-NN with k=3, weights=distance
Validation Accuracy: 0.8760
Training k-NN with k=5, weights=uniform
Validation Accuracy: 0.8733
Training k-NN with k=5, weights=distance
Validation Accuracy: 0.8733
Training k-NN with k=7, weights=uniform
Validation Accuracy: 0.8787
Training k-NN with k=7, weights=distance
Validation Accuracy: 0.8733
Training k-NN with k=9, weights=uniform
Validation Accuracy: 0.8787
Training k-NN with k=9, weights=distance
Validation Accuracy: 0.8760
Training k-NN with k=11, weights=uniform
Validation Accuracy: 0.8733
Training k-NN with k=11, weights=distance
Validation Accuracy: 0.8706

Best k-NN Model:
Best k: 7
Best weighting: uniform
Best validation accuracy: 0.8787061994609164
Best knn model saved


Correct predictions: 323
Unknown predictions: 5
Total samples: 371
